# Statistical quality control in anaylizing read sequencing data

## 1. Required libraries

**Biopython** libraries ([Cock et al., 2009](#cock_et_al)) are required in order to load and preprocessing raw data, [**numpy**](https://docs.scipy.org/doc/numpy/reference/) and [**pandas**](https://pandas.pydata.org/pandas-docs/stable/reference/index.html) are used to arrange data frames previously to data analyses, and [**bokeh**](https://bokeh.pydata.org/en/latest/docs/reference.html) is the Python module that we have studied to improve the visualizations

In [1]:
import os
import sys
from argparse import ArgumentParser
import logging
#Use regular expressions for extract the read id
import re

#Biopython module imports for loading and transforming sequence files
from Bio import SeqIO
from Bio.SeqUtils import GC

#Numpy and pandas to manage the data
import numpy as np
import pandas as pd

#Bokeh library
from bokeh.plotting import figure, save, output_file, show
from bokeh.layouts import gridplot, layout, row, column
from bokeh.models import Range1d, ColumnDataSource
#from version import __version__
from bokeh.io import output_notebook

## 2. Basic methods

The NanoDJ project ([Rodríguez-Pérez et al., 2019](#rodri_et_al)) offers several visualizations to analyze the quailty control of ONT reads. In this notebook, we improve these visualizations by using the **bokeh** module and several methods implemented in [**Nanoplot**](https://github.com/wdecoster/NanoPlot) project ([Wouter De Coster et al., 2018](#wouter_et_al))

In [2]:
#Function to build a pandas dataframe from a FASTA or FASTQ file
def read_sequence_file(filename, format):
    rows = []
    columns = ['id', 'sequence', 'read_length', 'gc_content']
    if(format == 'fastq'):
        columns.append('avg_quality')
    #Building each row with the read and its features
    for seq_record in SeqIO.parse(filename, format):
        if(format == 'fastq'):
            quality_score = np.mean(seq_record.letter_annotations["phred_quality"])
        read_id = seq_record.description
        if(format == 'fastq'):
            rows.append([read_id, seq_record.seq, len(seq_record.seq), GC(seq_record.seq), quality_score])
        else:
            rows.append([read_id, seq_record.seq, len(seq_record.seq), GC(seq_record.seq)])  
        
    df = pd.DataFrame(rows, columns=columns)
    return df


In [3]:
# ----- Place the path to your reads file here ----- #
seq_df = read_sequence_file('reads_agalactiae.fastq', 'fastq')

In [4]:
seq_df.describe()

,read_length,gc_content,avg_quality
count,3720.000000,3720.000000,3720.000000
mean,9366.188978,46.717829,11.024837
std,22288.676094,10.470068,5.260270
min,5.000000,0.000000,1.800000
25%,975.750000,37.198356,6.680380
50%,3057.500000,47.978136,8.093066
75%,9046.500000,54.879056,16.449813
max,388351.000000,90.000000,22.241614


In [5]:
def length_histogram(fqin):
    '''
    Create a histogram, and return the bin edges of the bin containing the most reads
    '''
    lengths = get_lengths(fqin)
    hist, edges = np.histogram(lengths, bins='auto')
    
    data = pd.DataFrame({'freq': hist, 
                       'left': edges[:-1], 
                       'right': edges[1:]})
    # Add a column showing the extent of each interval
    data['interval'] = ['from %d to %d ' % (left, right) for left, right in zip(data['left'], data['right'])]

    src = ColumnDataSource(data)
    #src.data.keys()

    hist_norm = figure()
    hist_norm.quad(source=src,
        top='freq',
        bottom=0,
        left='left',
        right='right',
        fill_color="#036564",
        line_color="#033649")
    hist_norm.xaxis[0].formatter.use_scientific = False
    return data,hist_norm



def get_lengths(fastq):
    '''
    Loop over the fastq file, extract length of sequences
    '''
    return np.array([len(record) for record in SeqIO.parse(fastq, "fastq")])


def per_base_sequence_content_and_quality(head_seq, head_qual, tail_seq, tail_qual, rna=False):
    seq_plot_left = plot_nucleotide_diversity(head_seq, rna=rna)
    seq_plot_right = plot_nucleotide_diversity(tail_seq, invert=True, rna=rna)
    qual_plot_left = plot_qual(head_qual)
    qual_plot_right = plot_qual(tail_qual, invert=True)
    logging.info("Per base sequence content and quality completed.")
    return [seq_plot_left, seq_plot_right], [qual_plot_left, qual_plot_right]


def get_bin(fq, size_range):
    '''
    Loop over the fastq file
    Extract list of nucleotides and list of quality scores in tuples in list
    Only select those reads of which the length is within the size range
    '''
    logging.info("Extracting nucleotides and quality scores.")
    return [(list(rec.seq)[:size_range],
             list(rec.letter_annotations["phred_quality"])[:size_range],
             list(rec.seq[-1 * size_range:]),
             list(rec.letter_annotations["phred_quality"])[-1 * size_range:])
            for rec in SeqIO.parse(fq, "fastq") if len(rec) >= size_range * 2]


def plot_nucleotide_diversity(seqs, invert=False, rna=False):
    x_length = len(seqs[0])
    if invert:
        p = figure(x_range=Range1d(start=x_length, end=0))
    else:
        p = figure()
    p.grid.grid_line_alpha = 0.3
    numreads = len(seqs)
    for nucl, color in zip(['A', 'U' if rna else 'T', 'G', 'C'],
                           ['green', 'pink' if rna else 'red', 'black', 'blue']):
        if invert:
            p.xaxis.axis_label = 'Position in read from end'
            p.line(
                x=range(x_length),
                y=list(reversed(np.array([pos.count(nucl) / numreads for pos in zip(*seqs)]))),
                color=color,
                legend=nucl)
        else:
            p.xaxis.axis_label = 'Position in read from start'
            p.line(
                x=range(x_length),
                y=np.array([pos.count(nucl) / numreads for pos in zip(*seqs)]),
                color=color,
                legend=nucl)
    p.yaxis.axis_label = 'Frequency of nucleotide in read'
    return p


def get_qual_per_pos(quallist):
    '''
    Plot average quality per position
    params:
    -quallist: List of the qualities per read.
    -sizeRange: Range of the first or last bases in the reads.
    '''
    mean_qual_per_pos = []
    # Extract the first positions to the last positions.
    for position in range(len(quallist[0])):
        # Make a list with quality per pos in all the reads in quallist.
        pos_x_list = [quality[position] for quality in quallist]
        # Calculate the mean quality of the position and append it to a list.
        mean_qual_per_pos.append(np.mean(pos_x_list))
    return mean_qual_per_pos


def plot_qual(quallist, invert=False):
    '''
    Create a FastQC-like "￼Per base sequence quality￼" plot
    Plot average quality per position
    zip will stop when shortest read is exhausted
    '''
    mean_quallist = get_qual_per_pos(quallist)
    x_length = len(mean_quallist)
    if invert:
        p = figure(x_range=Range1d(start=x_length, end=0))
        p.grid.grid_line_alpha = 0.3
        p.line(x=range(x_length),
               y=list(reversed(mean_quallist)),
               color='orange')
        p.xaxis.axis_label = 'Position in read from end'
    else:
        p = figure()
        p.grid.grid_line_alpha = 0.3
        p.line(x=range(x_length),
               y=mean_quallist,
               color='orange')
        p.xaxis.axis_label = 'Position in read from start'
    p.yaxis.axis_label = 'Mean quality score of base calls'
    return p

In [6]:
logging.basicConfig(
        format='%(asctime)s %(message)s',
        filename=os.path.join(".","NanoQC.log"),
        level=logging.INFO)
logging.info("NanoQC started.")

In [7]:
fq_input="reads_agalactiae.fastq"
size_range=500
data, hist = length_histogram(fqin=fq_input)
fq = get_bin(fq=fq_input, size_range=size_range)
data

,freq,left,right,interval
0,992,5.000000,1046.142091,from 5 to 1046
1,572,1046.142091,2087.284182,from 1046 to 2087
2,315,2087.284182,3128.426273,from 2087 to 3128
3,236,3128.426273,4169.568365,from 3128 to 4169
4,179,4169.568365,5210.710456,from 4169 to 5210
5,162,5210.710456,6251.852547,from 5210 to 6251
6,145,6251.852547,7292.994638,from 6251 to 7292
7,114,7292.994638,8334.136729,from 7292 to 8334
8,106,8334.136729,9375.278820,from 8334 to 9375
9,78,9375.278820,10416.420912,from 9375 to 10416


In [8]:
if len(fq) == 0:
        logging.critical(
            "No reads with a higher length of {}.".format(size_range * 2))
        logging.info("Exiting...")
else:
        logging.info(("Using {} reads with a minimum length of {}bp for plotting")
                     .format(len(fq), size_range * 2))
        logging.info("Creating plots...")
        seq_plots, qual_plots = per_base_sequence_content_and_quality(
            head_seq=[dat[0] for dat in fq],
            head_qual=[dat[1] for dat in fq],
            tail_seq=[dat[2] for dat in fq],
            tail_qual=[dat[3] for dat in fq],
            rna=False)
        #output_file(os.path.join(args.outdir, "nanoQC.html"), title="nanoQC_report")
        grid_plot=gridplot(children=[[hist], seq_plots, qual_plots],
                      plot_width=400,
                      plot_height=400)
        logging.info("Finished!")

In [9]:
output_notebook()

Loading BokehJS ...

In [10]:
from bokeh.models import Button, Toolbar, ToolbarBox
from bokeh.models.tools import HoverTool, WheelZoomTool, PanTool, CrosshairTool, LassoSelectTool, BoxZoomTool,ResetTool

hover = HoverTool(tooltips = [('Interval', '@interval'),
                             ('Num of reads', '@freq')])
hist.add_tools(hover)
hist.width=800
hist.height=400
show(gridplot(children=[[hist]], plot_width=800, plot_height=400))

In [11]:
from bokeh.models import Button, Toolbar, ToolbarBox
from bokeh.models.tools import HoverTool, WheelZoomTool, PanTool, CrosshairTool, LassoSelectTool, BoxZoomTool,ResetTool

tools = [HoverTool(), BoxZoomTool(), ResetTool()]
#fig = Figure(toolbar_location=None)
#fig.tools= tools
toolBarBox = ToolbarBox()
toolBarBox.toolbar = Toolbar(tools=tools)
toolBarBox.toolbar_location = "below"
show(gridplot(children=[[seq_plots[0]]], plot_width=800, plot_height=400))
show(gridplot(children=[[seq_plots[1]]], plot_width=800, plot_height=400))

In [12]:
tools = [HoverTool(), BoxZoomTool(), ResetTool()]
#fig = Figure(toolbar_location=None)
#fig.tools= tools
toolBarBox = ToolbarBox()
toolBarBox.toolbar = Toolbar(tools=tools)
toolBarBox.toolbar_location = "below"
show(gridplot(children=[[qual_plots[0]]], plot_width=800, plot_height=400))
show(gridplot(children=[[qual_plots[1]]], plot_width=800, plot_height=400))

## 3. Summary statistics

We have used the [**nanomath**](https://github.com/wdecoster/nanomath) and [**nanoget**](https://github.com/wdecoster/nanoget) programs developed by [Wouter De Coster et al., 2018](#wouter_et_al) to make a basic summary statistics table

In [13]:
import nanomath    #pip install nanomath

In [14]:
def extract_from_fastq(fq):
    """Extract metrics from a fastq file.
    Return average quality and read length
    """
    for rec in SeqIO.parse(fq, "fastq"):
        yield nanomath.ave_qual(rec.letter_annotations["phred_quality"]), len(rec)
        

def reduce_memory_usage(df):
    """reduce memory usage of the dataframe
    - convert runIDs to categorical
    - downcast ints and floats
    """
    usage_pre = df.memory_usage(deep=True).sum()
    if "runIDs" in df:
        df.loc[:, "runIDs"] = df.loc[:, "runIDs"].astype("category")
    df_int = df.select_dtypes(include=['int'])
    df_float = df.select_dtypes(include=['float'])
    df.loc[:, df_int.columns] = df_int.apply(pd.to_numeric, downcast='unsigned')
    df.loc[:, df_float.columns] = df_float.apply(pd.to_numeric, downcast='float')
    usage_post = df.memory_usage(deep=True).sum()
    logging.info("Reduced DataFrame memory usage from {}Mb to {}Mb".format(
        usage_pre / 1024**2, usage_post / 1024**2))
    if usage_post > 4e9 and "readIDs" in df:
        logging.info("DataFrame of features is too big, dropping read identifiers.")
        return df.drop(["readIDs"], axis=1, errors="ignore")
    else:
        return df
    
def process_fastq_plain(fastq, **kwargs):
    """Combine metrics extracted from a fastq file."""
    logging.info("Nanoget: Starting to collect statistics from plain fastq file.")
    inputfastq = open(fastq, 'r')
    return reduce_memory_usage(pd.DataFrame(
        data=[res for res in extract_from_fastq(inputfastq) if res],
        columns=["quals", "lengths"]
    ).dropna())

def make_stats(datadf, settings, suffix):
    statsfile = "NanoStats" + suffix + ".txt"
    nanomath.write_stats(
        datadfs=[datadf],
        outputfile=statsfile)
    logging.info("Calculated statistics")
    return statsfile

def chunks(values, chunks):
    if values:
        chunksize = int(len(values) / chunks)
        return([' '.join(values[i:i + chunksize]) for i in range(0, len(values), chunksize)])
    else:
        return [" "] * chunks
    
def stats2html(statsf):
    df = pd.read_csv(statsf, sep=':', header=None, names=['feature', 'value'])
    values = df["value"].str.strip().str.replace('\t', ' ').str.split().replace(np.nan, '')
    num = len(values[0]) or 1
    v = [chunks(i, num) for i in values]
    return pd.DataFrame(v, index=df["feature"]).to_html(header=False)


In [15]:
from IPython.core.display import display, HTML
datadf=process_fastq_plain('reads_agalactiae.fastq')
statsfile = [make_stats(datadf, settings="", suffix="")]
display(HTML(stats2html(statsfile[0])))

feature,
General summary,
Mean read length,"9,366.2"
Mean read quality,7.9
Median read length,"3,057.5"
Median read quality,6.4
Number of reads,"3,720.0"
Read length N50,"27,499.0"
Total bases,"34,842,223.0"
"Number, percentage and megabases of reads above quality cutoffs",
>Q5,3302 (88.8%) 34.1Mb


In [16]:
#To limit the lines in the table, we have to access to the html file and cut them manually.
display(HTML(stats2html(statsfile[0])))

feature,
General summary,
Mean read length,"9,366.2"
Mean read quality,7.9
Median read length,"3,057.5"
Median read quality,6.4
Number of reads,"3,720.0"
Read length N50,"27,499.0"
Total bases,"34,842,223.0"
"Number, percentage and megabases of reads above quality cutoffs",
>Q5,3302 (88.8%) 34.1Mb


## 4. References

<a id='cock_et_al'></a> Cock, P. J., Antao, T., Chang, J. T., Chapman, B. A., Cox, C. J., Dalke, A., Friedberg, I., Hamelryck, T., Kauff, F., Wilczynski, B., de Hoon, M. J. Biopython: freely available Python tools for computational molecular biology and bioinformatics. Bioinformatics, Volume 25, Issue 11, 1 June 2009, Pages 1422–1423. DOI http://dx.doi.org/10.1093/bioinformatics/btp163

<a id='wouter_et_al'></a> Wouter De Coster, Svenn D’Hert, Darrin T Schultz, Marc Cruts, Christine Van Broeckhoven, NanoPack: visualizing and processing long-read sequencing data, Bioinformatics, Volume 34, Issue 15, 01 August 2018, Pages 2666–2669, https://doi.org/10.1093/bioinformatics/bty149

<a id='rodri_et_al'></a>Rodríguez-Pérez H, Hernández-Beeftink T, Lorenzo-Salazar JM, Roda-García JL, Pérez-González CJ, Colebrook M, Flores C. NanoDJ: A Dockerized Jupyter Notebook for Interactive Oxford Nanopore MinION Sequence Manipulation and Genome Assembly. BMC Bioinformatics (2019 20:234) https://doi.org/10.1186/s12859-019-2860-z
